# Creating dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [3]:
DATADIR = '../datasets/Classification/Dataset - 3 dias/Images'
CATEGORIES = ["Normal", "Anormal", "Morta"]

##loading images for classification
for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)

In [4]:
def create_training_data():
    training_data = []
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
            img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
            ## add images to training data with its label
            training_data.append([img_array, class_num])
    return training_data
            
training_data = create_training_data()
            

In [5]:
print(len(training_data))

1101


In [6]:
#mix data for better learning
import random
random.shuffle(training_data)

In [7]:
images = []
labels = []

for image, label in training_data:
    images.append(image)
    labels.append(label)

In [8]:
##saving dataset
import pickle
DATASET_PATH = "../datasets/Classification/Dataset - 3 dias/"

pickle_out = open(os.path.join(DATASET_PATH, "3_days_images.pickle"), "wb")
pickle.dump(images, pickle_out)
pickle_out.close()

pickle_out = open(os.path.join(DATASET_PATH, "3_days_labels.pickle"), "wb")
pickle.dump(labels, pickle_out)
pickle_out.close()



# Creating classificatin model
##### Following this tutorial
       www.youtube.com/watch?v=WvoLTXIjBYU

In [75]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import keras



In [76]:
#reading the dataset
images_raw = open(os.path.join(DATASET_PATH, "3_days_images.pickle" ), "rb")
images = pickle.load(images_raw)

labels_raw = open(os.path.join(DATASET_PATH, "3_days_labels.pickle" ), "rb")
labels = pickle.load(labels_raw)

#reshape images
IMG_SIZE = 256

for i in range(0, len(images)):
    images[i] = cv2.resize(images[i], dsize=(IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC)
    images[i] = np.array(images[i])
    labels[i] = np.array(labels[i])

images = np.array(images)
labels = np.array(labels)
labels_test = labels.reshape(-1,1)

##one hot 
import pandas as pd

labels = pd.get_dummies(labels)
labels = pd.DataFrame.to_numpy(labels)

#normalizing images    
images = tf.keras.utils.normalize(images, axis=0, order=2)

In [77]:
print(labels[0].shape)

(3,)


In [84]:
BATCH_SIZE = 32
EPOCHS = 25
TRAIN_TEST_SPLIT = 0.25

model = Sequential()

model.add(   Conv2D(64, (3,3), input_shape=images.shape[1:])   )
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))

model.add(Dense(3))
model.add(Activation("sigmoid"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

model.fit(images, labels, batch_size = BATCH_SIZE, epochs = EPOCHS, validation_split=TRAIN_TEST_SPLIT)

Epoch 1/25
26/26 [==============================] - 55s 2s/step - loss: 0.6629 - accuracy: 0.7145 - val_loss: 0.3047 - val_accuracy: 0.8696
Epoch 2/25
26/26 [==============================] - 47s 2s/step - loss: 0.2458 - accuracy: 0.9148 - val_loss: 0.2712 - val_accuracy: 0.8841
Epoch 3/25
26/26 [==============================] - 47s 2s/step - loss: 0.2257 - accuracy: 0.9142 - val_loss: 0.2944 - val_accuracy: 0.8877
Epoch 4/25
26/26 [==============================] - 47s 2s/step - loss: 0.1914 - accuracy: 0.9219 - val_loss: 0.3704 - val_accuracy: 0.8478
Epoch 5/25
26/26 [==============================] - 49s 2s/step - loss: 0.1659 - accuracy: 0.9149 - val_loss: 0.3664 - val_accuracy: 0.8696
Epoch 6/25
26/26 [==============================] - 50s 2s/step - loss: 0.1345 - accuracy: 0.9390 - val_loss: 0.3299 - val_accuracy: 0.8804
Epoch 7/25
26/26 [==============================] - 49s 2s/step - loss: 0.1173 - accuracy: 0.9437 - val_loss: 0.3960 - val_accuracy: 0.8696
Epoch 8/25
26/26 [==

In [79]:
model.evaluate(images, labels)

35/35 [==============================] - 13s 360ms/step - loss: 0.1799 - accuracy: 0.9328


[0.1798994541168213, 0.9327883720397949]